Importando as bibliotecas

In [ ]:
import os
import requests
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

Importando os dados

In [ ]:
diretorio_principal = os.getcwd()
subdiretorio_dados = os.path.join(diretorio_principal, "Dados")

# Criar o diretório se não existir
os.makedirs(subdiretorio_dados, exist_ok=True)

In [ ]:
anos = range(2008, 2024)
formato = "csv"
url_base = "http://www.camara.leg.br/cotas/"

for ano in anos:
    download = requests.get(url_base + f"Ano-{ano}.{formato}.zip")
    caminho_zip = os.path.join(subdiretorio_dados, f"Ano-{ano}.{formato}.zip")

    with open(caminho_zip, "wb") as file:
        file.write(download.content)

    # Extrair o conteúdo do arquivo ZIP
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(subdiretorio_dados)

    # # Remover o arquivo ZIP após extração (opcional)
    # os.remove(caminho_zip)

In [ ]:
# Voltar ao diretório principal
os.chdir(diretorio_principal)

# Criar um DataFrame com os dados extraídos
despesas_totais = []

for arquivo_csv in os.listdir(subdiretorio_dados):
    if arquivo_csv.endswith('.csv') and not arquivo_csv.startswith("."):
        caminho_csv = os.path.join(subdiretorio_dados, arquivo_csv)
        despesas = pd.read_csv(caminho_csv, sep=";", encoding='UTF-8', low_memory=False)
        despesas_totais.append(despesas)

# Concatenar todos os DataFrames em um único DataFrame
despesas_completo = pd.concat(despesas_totais, ignore_index=True)

In [ ]:
despesas_completo

Selecionando colunas de interesse

In [ ]:
# Listar colunas
despesas_completo.columns

In [ ]:
# Selecionar colunas
dataset = despesas_completo.loc[:,['sgUF', 'sgPartido', 'txtDescricao', 'datEmissao', 'vlrLiquido']]

In [ ]:
# Visualizar dados de estudo
dataset

Conferindo dados Selecionados

In [ ]:
dataset.head(10)

In [ ]:
dataset.tail(10)

In [ ]:
dataset.shape

In [ ]:
dataset['vlrLiquido'].describe()

In [ ]:
dataset[['sgUF', 'sgPartido', 'txtDescricao']].describe()

In [ ]:
dataset.info()

Alterar tipos de dados

In [ ]:
dataset['datEmissao'] = pd.to_datetime(dataset['datEmissao'], errors='coerce')

Limpeza de dados

In [ ]:
#Remover duplicatas
dataset = dataset.drop_duplicates(keep="first")

In [ ]:
#Remover valores
#dataset = dataset.loc[dataset['coluna'] == 'valor']

In [ ]:
#Remover colunas desnecessárias
#dataset = dataset.drop(columns=['coluna1', 'coluna2'])

Dados faltantes

In [ ]:
#Deletar linhas com dados faltantes
dataset = dataset.dropna()

In [ ]:
#Imputar valores
""" imputer = SimpleImputer(minssing_valeues = np.nan, strategy='mean')
imputer.fit(dataset[:,:])
imputer.transform(dataset[:, :]) """

In [ ]:
#Substituir valores
# dataset.replace(old_value, new_value, inplace=True)

Selecionar Foco do estudo

In [ ]:
dataset[['sgUF', 'sgPartido', 'txtDescricao']].describe()

In [ ]:
dataset = dataset[dataset['txtDescricao'] == 'COMBUSTÍVEIS E LUBRIFICANTES.']

In [ ]:
dataset.columns

Medidas de tendência central

In [ ]:
# Média geral
dataset['vlrLiquido'].mean()


In [ ]:
# Média por estado
dataset.groupby('sgUF')['vlrLiquido'].mean()

In [ ]:
# Média por partido
dataset.groupby('sgPartido')['vlrLiquido'].mean()

In [ ]:
# Moda geral
dataset['vlrLiquido'].mode()

In [ ]:
# Moda por estado
dataset.groupby('sgUF')['vlrLiquido'].mode()

In [ ]:
# Moda por partido
dataset.groupby('sgPartido')['vlrLiquido'].mode()


In [ ]:
# Mediana geral
dataset['vlrLiquido'].median()

In [ ]:
# Mediana por estado
dataset.groupby('sgUF')['vlrLiquido'].median()

In [ ]:
# Mediana por partido
dataset.groupby('sgPartido')['vlrLiquido'].median()

Medidas de dispersão

In [ ]:
# Amlitude total geral
dataset['vlrLiquido'].max() - dataset['vlrLiquido'].min()

In [ ]:
# Amlitude total por estado
dataset.groupby('sgUF')['vlrLiquido'].max() - dataset.groupby('sgUF')['vlrLiquido'].min()

In [ ]:

# Amlitude total por partido
dataset.groupby('sgPartido')['vlrLiquido'].max() - dataset.groupby('sgPartido')['vlrLiquido'].min()

In [ ]:
# Intervalo interquartílico geral
Q1 = dataset['vlrLiquido'].quantile(0.25)
Q3 = dataset['vlrLiquido'].quantile(0.75)
IQ = Q3-Q1
IQ

In [ ]:

# Intervalo interquartílico por estado
Q1 = dataset.groupby('sgUF')['vlrLiquido'].quantile(0.25)
Q3 = dataset.groupby('sgUF')['vlrLiquido'].quantile(0.75)
IQ = Q3-Q1
IQ

In [ ]:

# Intervalo interquartílico por partido
Q1 = dataset.groupby('sgPartido')['vlrLiquido'].quantile(0.25)
Q3 = dataset.groupby('sgPartido')['vlrLiquido'].quantile(0.75)
IQ = Q3-Q1
IQ


In [ ]:

# Variância geral
dataset['vlrLiquido'].var()

In [ ]:

# Variância por estado
dataset.groupby('sgUF')['vlrLiquido'].var()


In [ ]:
# Variância por partido
dataset.groupby('sgPartido')['vlrLiquido'].var()

In [ ]:

# Desvio padrão geral
dataset['vlrLiquido'].std()

In [ ]:
# Desvio padrão por estado
dataset.groupby('sgUF')['vlrLiquido'].std()

In [ ]:
# Desvio padrão por partido
dataset.groupby('sgPartido')['vlrLiquido'].std()

Medidas de Assemetria

Outliers

In [ ]:
# Diagrama de Tukey (boxplot)
gridsize_boxplot = (4, 1)
fig_boxplot = plt.figure(figsize=(12, 8))

positions = ['sgUF', 'sgPartido', 'vlrLiquido']

for i, position in enumerate(positions):
    boxplot = plt.subplot2grid(gridsize_boxplot, (i, 0))
    boxplot.set_title(position, fontsize=8)
    sns.boxplot(x=position, data=dataset)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de dispersão
gridsize_scatter = (4, 1)
fig_scatter = plt.figure(figsize=(12, 8))

positions = ['sgUF', 'sgPartido', 'vlrLiquido']

for i, position in enumerate(positions):
    scatterplot = plt.subplot2grid(gridsize_scatter, (i, 0))
    scatterplot.set_title(position, fontsize=8)
    
    if dataset[position].dtype == 'object':
        sns.countplot(x=position, data=dataset, ax=scatterplot)
    else:
        sns.scatterplot(x=position, y='vlrLiquido', data=dataset, ax=scatterplot)

plt.tight_layout()
plt.show()

In [ ]:
# Histograma
gridsize_barplot = (4, 1)
fig_barplot = plt.figure(figsize=(12, 8))

positions = ['sgUF', 'sgPartido', 'txtDescricao', 'vlrLiquido']

for i, position in enumerate(positions):
    barplot = plt.subplot2grid(gridsize_barplot, (i, 0))
    barplot.set_title(position, fontsize=8)
    
    if dataset[position].dtype == 'object':
        sns.countplot(x=position, data=dataset, ax=barplot)
    else:
        sns.histplot(x=position, data=dataset, ax=barplot)

plt.tight_layout()
plt.show()

In [ ]:
# Remover outliers - Winsorization

""" cols = list(dataset.columns)
cols.remove('col')

winsorization_flooring = 0.05
winsorization_capping = 1 - winsorization_flooring

not_selected_data_flooring = dataset[cols] <= dataset[cols].quantile(winsorization_flooring)
not_selected_data_capping = dataset[cols] >= dataset[cols].quantile(winsorization_capping)

not_selected_data_all = (not_selected_data_flooring & not_selected_data_capping)

data_index = dataset[cols][not_selected_data_all].index
dataset[cols].drop(data_index, inplace=True """

Regressão Linear

In [ ]:
dataset = dataset.loc[:,['datEmissao', 'vlrLiquido']]

In [ ]:
#Parcionando dados

In [ ]:
# Criar modelo
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Predição
predicted_values = model.predict(X_test)

In [ ]:
# Análise da regressão